In [2]:
import pandas as pd
import numpy as np 
import sys
sys.path.append('../../Function')
import function
from sklearn.metrics import mean_absolute_error
from keras.layers import Dense, Dropout
from keras.models import Sequential


In [3]:
df = pd.read_excel('../../../site_info_ver_5.xlsx')

In [21]:
dataset = df.copy()
sites = dataset.Site_Name.unique()
site_name = 'The Masada'
target = 'Israelis_Count'
target_title = 'Israelis'
dataset = dataset.loc[dataset.Site_Name==site_name]
# dataset = dataset.loc[dataset[target]<2000]


# dataset = function.remove_pollution_site(dataset)
# dataset = dataset.drop(dataset.filter(regex='pm').columns, axis=1)
# dataset = dataset.drop(dataset.filter(regex='Ashkelon').columns, axis=1)
dataset = dataset.drop(['Total','Tourists_Count'],axis=1)
dataset = dataset.drop(dataset.filter(regex='nox').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='so2').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='pm2.5').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='Ashkelon').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='Season').columns, axis=1)

print('model on site',site_name)
print('Focus on ',target_title)

# dataset = function.remove_outliers(dataset, target)

dataset = function.remove_unique_one(dataset)
dataset = function.remove_high_corr(dataset, target,0.6)
dataset = function.split_date(dataset)
# dataset = function.get_weekday(dataset)
dataset = function.last_year_entries_info(dataset,target)
# dataset.set_index("Date",inplace=True)
dataset.drop("Date",axis=1,inplace=True)
# dataset = dataset.astype({'day': 'category'})
# dataset = dataset.astype({'month': 'category'})
# dataset = dataset.astype({'year': 'category'})
dataset = dataset.drop('day', axis=1)
dataset = dataset.drop('month', axis=1)
dataset = dataset.drop('year', axis=1)
dataset = dataset.drop('is_jewish_vacation', axis=1)
dataset = dataset.drop('is_muslims_holiday', axis=1)

print(dataset.dtypes)
print(dataset.shape)
dataset = pd.get_dummies(dataset)

print('shape of dataset',dataset.shape)
print('features :',dataset.columns)
dataset[['Israelis_Count']].describe()

model on site The Masada
Focus on  Israelis
Add Last year visitors Successfully
is_weekend                       int64
operations                       int64
is_jewish_holiday                int64
pm10                           float64
Temperature                    float64
is_HeatWave                      int64
Haifa_pm10                     float64
Beer-Sheva_pm10                float64
is_Site_exceeded_pm10            int64
Tel_Aviv-Yafo_pm10_exceeded      int64
Jerusalem_pm10_exceeded          int64
Haifa_pm10_exceeded              int64
Last_year_visitors             float64
Israelis_Count                   int64
dtype: object
(1495, 14)
shape of dataset (1495, 14)
features : Index(['is_weekend', 'operations', 'is_jewish_holiday', 'pm10', 'Temperature',
       'is_HeatWave', 'Haifa_pm10', 'Beer-Sheva_pm10', 'is_Site_exceeded_pm10',
       'Tel_Aviv-Yafo_pm10_exceeded', 'Jerusalem_pm10_exceeded',
       'Haifa_pm10_exceeded', 'Last_year_visitors', 'Israelis_Count'],
      dtype='ob

,Israelis_Count
count,1495.000000
mean,1577.650836
std,790.851670
min,61.000000
25%,959.000000
50%,1446.000000
75%,2106.500000
max,3982.000000


In [5]:
#change more columns:


In [22]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = dataset.drop(target,axis=1)
y = dataset[target]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=312148513)

X_train_scaler = MinMaxScaler()
X_test_scaler = MinMaxScaler()

X_train_scaled = X_train_scaler.fit_transform(X_train)
X_test_scaled = X_test_scaler.fit_transform(X_test)

y_train_scaled = np.log(y_train+0.001)
y_test_scaled = np.log(y_test+0.001)

dataset_train = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
dataset_test = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)



STDs

In [29]:
print("train STD " + str(np.std(y_train)))
print("test STD " + str(np.std(y_test)))

train STD 792.9475515905618
test STD 779.7309992145478


MLR Test

In [23]:
from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(X_train_scaled,y_train)

prediction =mlr.predict(X_test_scaled)

res = pd.DataFrame(
    data={
        'Prediction':prediction,
        'Actual': y_test.values
    },
    index=y_test.index
)

print('rmse',function.get_rmse(res.Prediction, res.Actual))
print('std',np.std(res.Actual))

coef = sorted( list(zip(np.round(mlr.coef_,3).T,X_train.columns)))
coef

rmse 708.7708573423363
std 779.7309992145478


[(-496.496, 'Beer-Sheva_pm10'),
 (-293.846, 'Temperature'),
 (-233.077, 'is_HeatWave'),
 (-105.908, 'is_weekend'),
 (-63.673, 'operations'),
 (5.77, 'Haifa_pm10_exceeded'),
 (9.242, 'is_Site_exceeded_pm10'),
 (15.868, 'Jerusalem_pm10_exceeded'),
 (35.102, 'pm10'),
 (85.762, 'Tel_Aviv-Yafo_pm10_exceeded'),
 (149.01, 'Haifa_pm10'),
 (304.292, 'is_jewish_holiday'),
 (1724.364, 'Last_year_visitors')]

MLR Train

In [24]:
prediction =mlr.predict(X_train_scaled)

res = pd.DataFrame(
    data={
        'Prediction':prediction,
        'Actual': y_train.values
    },
    index=y_train.index
)

print('rmse',function.get_rmse(res.Prediction, res.Actual))
print('std',np.std(res.Actual))

coef = sorted( list(zip(np.round(mlr.coef_,3).T,X_train.columns)))
coef

rmse 685.961788944731
std 792.9475515905618


[(-496.496, 'Beer-Sheva_pm10'),
 (-293.846, 'Temperature'),
 (-233.077, 'is_HeatWave'),
 (-105.908, 'is_weekend'),
 (-63.673, 'operations'),
 (5.77, 'Haifa_pm10_exceeded'),
 (9.242, 'is_Site_exceeded_pm10'),
 (15.868, 'Jerusalem_pm10_exceeded'),
 (35.102, 'pm10'),
 (85.762, 'Tel_Aviv-Yafo_pm10_exceeded'),
 (149.01, 'Haifa_pm10'),
 (304.292, 'is_jewish_holiday'),
 (1724.364, 'Last_year_visitors')]